In [46]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_algorithms.utils import algorithm_globals
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_aer import AerSimulator

In [47]:
import numpy as np

# Define number of servers (N), services (S) and users (U)
N = 5
S = 2
U = 10

# Generate random user-server connections (ngb), user service requests (service) and server states (server_state)
np.random.seed(0)
ngb = np.random.randint(0, 2, (U, N))
service = np.random.randint(0, S, U)
server_state = np.random.rand(N, S)

# Define server capacities
server_capacity = np.zeros((N, S))
for server_id in range(N):
    state = server_state[server_id]
    u1 = int(state[0] // 5) + 1
    u2 = int((state[0] + 1) - (u1 - 1) * 5)
    server_capacity[server_id][0] = u1 * 100
    server_capacity[server_id][1] = u2 * 100

# Generate problem matrix (adjacency matrix)
problem_matrix = np.zeros((U * N, U * N))
for i in range(U):
    for j in range(N):
        if ngb[i, j] == 1:
            problem_matrix[i * N + j, i * N + j] = -server_capacity[j, service[i]]
            for k in range(j + 1, N):
                if ngb[i, k] == 1:
                    problem_matrix[i * N + j, i * N + k] = 2

In [48]:


def get_allocation_operator(problem_matrix):
    num_qubits = len(problem_matrix)
    pauli_list = []
    coeffs = []
    shift = 0

    for i in range(num_qubits):
        for j in range(i):
            if problem_matrix[i, j] != 0:
                z_p = np.zeros(num_qubits, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, np.zeros(num_qubits, dtype=bool))))
                coeffs.append(problem_matrix[i, j])
                shift += problem_matrix[i, j]

    for i in range(num_qubits):
        z_p = np.zeros(num_qubits, dtype=bool)
        z_p[i] = True
        pauli_list.append(Pauli((z_p, np.zeros(num_qubits, dtype=bool))))
        coeffs.append(1.0)
        shift += 1

    return SparsePauliOp(pauli_list, coeffs=coeffs), shift

qubit_op, offset = get_allocation_operator(problem_matrix)

In [49]:
print(qubit_op)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIII

In [59]:

# Define sampler
sampler = SamplerV2()
# Set random seed
algorithm_globals.random_seed = 10598

# Define optimizer
optimizer = COBYLA()

# Initialize QAOA
qaoa = QAOA( sampler=sampler, optimizer=optimizer, reps=2)

In [60]:
result = qaoa.compute_minimum_eigenvalue(qubit_op)



TypeError: SamplerV2.run() takes 2 positional arguments but 3 were given

In [61]:
# Sample most likely allocation
def sample_most_likely(state_vector):
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = [int(b) for b in bin(k)[2:].zfill(n)]
    return np.asarray(x)

allocation = sample_most_likely(result.eigenstate)
print("Allocation result:", allocation)

NameError: name 'result' is not defined